In [1]:
import rasterio
from rasterio.transform import array_bounds, from_bounds, from_origin
from rasterio.warp import Resampling
from os.path import join, isfile
import numpy as np

In [2]:
from pyflwdir.gis_utils import reggrid_area, affine_to_coords

In [3]:
# IN
wp_fn = r'/home/dirk/datasets/WorldPop/ppp_2010_1km_Aggregated.tif'
with rasterio.open(wp_fn, 'r') as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 43200, 'height': 18720, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0083333333, 0.0, -180.001249265,
       0.0, -0.0083333333, 84.00791653201003), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}


In [4]:
# src_tot_pop = 0
# with rasterio.open(wp_fn, 'r') as src:
#     for ji, wdw in src.block_windows(1):
#         pop_30sec = src.read(1, window=wdw)
#         src_tot_pop += np.nansum(pop_30sec[pop_30sec!=src.nodata])
# src_tot_pop
src_tot_pop = 6812530928.509598
dst_tot_pop_nearest = 6810460454.075542
dst_tot_pop_bilinear = 6812976189.401252
err_bil = np.abs(1-dst_tot_pop_bilinear/src_tot_pop)*100  
err_nn = np.abs(1-dst_tot_pop_nearest/src_tot_pop)*100
err_bil, err_nn

(0.006535910021199243, 0.030392147291258276)

In [5]:
# out
xmin, ymin, xmax, ymax = -180, -72, 180, 84
xsize, ysize = 0.005, 0.005
blockxsize, blockysize = 400, 400
wp2_fn = r'/home/dirk/datasets/WorldPop/ppp_2010_18arcsec_Aggregated_nearest.tif'

tot_pop = 0
# ppp to density 
wpd_fn = wp_fn.replace('.tif', '_density.tif')
with rasterio.open(wp_fn, 'r') as src:
    prof = src.profile.copy()
    prof.update(
        transform = from_origin(xmin, ymax, xsize, ysize),
        width = int((xmax-xmin) / xsize),
        height = int((ymax-ymin) / xsize),
        blockxsize = blockxsize,
        blockysize = blockysize,
    )
    with rasterio.open(wp2_fn, 'w', **prof) as dst:
        for ji, dst_wdw in dst.block_windows(1):
#             if not np.all(ji == (10, 10)): continue
            
            # get window properties dst (18sec)
            dst_transform = dst.window_transform(dst_wdw)
            dst_shape = dst_wdw.height, dst_wdw.width
            dst_bounds = array_bounds(*dst_shape, transform=dst_transform)
            
            # get src window  (30sec)
            src_wdw = rasterio.windows.round_window_to_full_blocks(
                window=rasterio.windows.from_bounds(*dst_bounds, transform=src.transform), 
                block_shapes=src.block_shapes
            )

            # read pop
            pop_30sec = src.read(1, window=src_wdw)
            
            # convert to density
            src_transform = src.window_transform(src_wdw)
            src_bounds = array_bounds(*pop_30sec.shape, transform=src_transform)
            src_lons, src_lats = affine_to_coords(src_transform, pop_30sec.shape[1], pop_30sec.shape[0])
            area_30sec = reggrid_area(src_lats, src_lons) #km
            dens_30sec = np.where(pop_30sec != src.nodata, pop_30sec / area_30sec, src.nodata)
            assert np.all(np.isfinite(dens_30sec))
            
            # resample
            dens_18sec = np.ones(dst_shape)*dst.nodata
            rasterio.warp.reproject(
                source=dens_30sec,
                destination=dens_18sec,
                resampling=Resampling.nearest,
                src_transform=src_transform,
                src_crs=src.crs,
                src_nodata=src.nodata,
                dst_transform=dst_transform,
                dst_height=dst_wdw.height, 
                dst_width=dst_wdw.width,
                dst_crs=dst.crs,
                dst_nodata=dst.nodata
            )
            assert np.all(np.isfinite(dens_18sec))
            # to pop per pixel
            dst_lons, dst_lats = affine_to_coords(dst_transform, dst_wdw.width, dst_wdw.height)
            area_18sec = reggrid_area(dst_lats, dst_lons)
            pop_18sec = np.where(dens_18sec!=dst.nodata, dens_18sec * area_18sec, dst.nodata)
            tot_pop += pop_18sec[pop_18sec!=src.nodata].sum()
            # write
            continue
            dst.write(pop_18sec.astype(np.float32), window=dst_wdw, indexes=1)
print(tot_pop)
print(tot_pop/src_tot_pop)

6810460454.075542
0.9996960785270874


# write to binary files for fortran impact routine

In [ ]:
import pandas as pd
mdir = r'/home/dirk/models/CaMa-Flood_v3.6.2/map/global_15min'
fn_regions = join(mdir,'hires', 'location.txt')
regions = pd.read_csv(fn_regions, delim_whitespace=True, index_col=0).T \
            .set_index('area').astype(float).to_dict(orient='index')

In [ ]:
import xarray as xr 

wp2_fn = r'/home/dirk/datasets/WorldPop/ppp_2010_18arcsec_Aggregated_bilinear.tif'

for area in regions:
    fn = join(mdir, 'hires', f'{area}.catmxy.tif')
    fn_out_bin = join(mdir, 'hires', f'{area}.worldpop')
    if isfile(fn_out_bin): continue
    print(area)
    with rasterio.open(fn, 'r') as like:
        bounds = like.bounds
        # xarray to deal with si2 which passes -180 line
        ds = xr.open_rasterio(wp2_fn, chunks={'x':400, 'y':400}).drop('band').squeeze()
        mv = ds.attrs['nodatavals'][0]
        w, s, e, n = bounds
        if e > 180 or w < -180:
            lon_org = np.copy(ds['x'].values)
            if e > 180:
                ds['x'] = xr.Variable('x', np.where(lon_org < 0, lon_org + 360, lon_org))
            else:
                ds['x'] = xr.Variable('x', np.where(lon_org > 0, lon_org - 360, lon_org))
            ds = ds.sortby('x')
        pop = ds.sel({'x':slice(w, e), 'y':slice(n, s)}).values
        ds.close()
        assert np.all(pop.shape == like.shape)
        np.where(pop!=mv, pop, 0).astype(np.float32).tofile(fn_out_bin) # mv zeros